# Training a Simple CNN

In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data.load_data import *
from training.cnn import ConvNet
from training.train import *
from training.eval import *
from data.convert_labels import *
import matplotlib.pyplot as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Create ImageDataset for training
train_annotations_file_path = 'data/real/01_20_160_120/train/01_20_160_120_train.csv'
train_img_dir_path = 'data/real/01_20_160_120/train'
test_annotations_file_path = 'data/real/01_20_160_120/test/01_20_160_120_test.csv'
test_img_dir_path = 'data/real/01_20_160_120/test'

# Can override arguments specifying class boundaries
target_transform = lambda target: convert_labels(target)

train_loader = load_data(train_annotations_file_path, train_img_dir_path, target_transform)
test_loader = load_data(test_annotations_file_path, test_img_dir_path, target_transform)

train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print(np.unique(train_labels))

In [ ]:
model = ConvNet(input_dims=(160, 120))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

results = train(model, criterion, optimizer, train_loader, epochs=10)

In [ ]:
iter_train_loss_list, train_acc_list = results
plt.plot(iter_train_loss_list)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.title('Loss history')
plt.show()

plt.plot(train_acc_list)
plt.xlabel('Epoch number')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()

In [ ]:
test_acc = evaluate_acc(model, test_loader)
print(test_acc)